https://github.com/hhhuang/CAG

## 第一版 (BERTScore結果可以，但BLEU結果不好，可能是因為模型多講與離題)以這個為主

In [ ]:
# =======================================
# Colab SETUP —— 安裝與權杖
# =======================================
!pip -q install transformers accelerate peft bitsandbytes datasets evaluate \
            bert_score sacrebleu>=2.3.0 sentencepiece

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2025.3.2 requires fsspec==2025.3.2, but you have fsspec 2024.12.0 which is incompatible.


In [ ]:
import os, json, torch, evaluate
from datasets import load_dataset, Dataset
from transformers import (AutoTokenizer, AutoModelForCausalLM,
                          TrainingArguments, Trainer,
                          DataCollatorForLanguageModeling,
                          BitsAndBytesConfig)
from peft import LoraConfig, get_peft_model
from functools import partial
from tqdm import tqdm

In [ ]:
from huggingface_hub import login

hf_token = "" # @param{type:“string”} 需替換
login(token=hf_token, add_to_git_credential=True)

!huggingface-cli whoami

Hsuan0929


In [ ]:
HF_TOKEN = os.getenv(hf_token)         # ⬅️ Colab > 「變數」加入個人 HF Token
#assert HF_TOKEN #"請在 Colab 環境變數中設定 HF_TOKEN"

MODEL_NAME   = "meta-llama/Llama-3.2-3B-Instruct"  # 或 3B 版，GPU 夠再換
JSON_PATH    = "/content/energy_chat.jsonl.txt"        # ⬅️ 您的對話檔
VAL_SPLIT    = 0.05                                # 驗證集比例
OUTPUT_DIR   = "/content/llama32_finetuned"
BATCH_SIZE   = 4
EPOCHS       = 2
LR           = 1e-4
MAX_LEN      = 512

In [ ]:
from datasets import load_dataset, Dataset
import json, re

# 1. 讀進 jsonl 檔（每行一個 JSON 物件）
raw_lines = [json.loads(l) for l in open("energy_chat.jsonl.txt", encoding="utf-8")]

# 2. 建成 HF Dataset
ds = Dataset.from_list(raw_lines)

# 3. 寫一個轉換函式：把 <|start_header_id|>user … assistant … 抽出
def split_text(example):
    txt = example["text"]
    user = re.search(r"user<\|end_header_id\|>\s*(.*?)\s*<\|eot_id\|>", txt, re.S)
    asst = re.search(r"assistant<\|end_header_id\|>\s*(.*?)\s*<\|eot_id\|>", txt, re.S)

    example["prompt"]   = user.group(1).strip() if user else ""
    example["response"] = asst.group(1).strip() if asst else ""
    return example

ds = ds.map(split_text, remove_columns=["text"])   # ← 轉完就只有 prompt/response

# 4. （可選）切分訓練/驗證集
dataset = ds.train_test_split(test_size=0.05, seed=42)
train_ds, val_ds = dataset["train"], dataset["test"]


Map:   0%|          | 0/137 [00:00<?, ? examples/s]

In [ ]:
'''
# =======================================
# 1. 讀取對話 JSON 並轉 HF Dataset
#    期待格式: [{"prompt": "...", "response": "..."}, ...]
# =======================================
with open(JSON_PATH, encoding="utf-8") as f:
    raw_data = json.load(f)

COLUMN_PROMPT   = "prompt"    # 如果欄位名稱不同就在這裡改
COLUMN_RESPONSE = "response"

dataset = Dataset.from_list(raw_data).train_test_split(test_size=VAL_SPLIT, seed=42)
train_ds, val_ds = dataset["train"], dataset["test"]
'''
# =======================================
# 2. 載入量化模型 + LoRA
# =======================================
bnb_cfg = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_use_double_quant=True
)

tokenizer = AutoTokenizer.from_pretrained(
    MODEL_NAME,
    token=HF_TOKEN,
    padding_side="right",
    use_fast=False)               # Llama tokenizer 通常不用 fast

# Llama 無預設 padding token → 這裡直接沿用 EOS
tokenizer.pad_token = tokenizer.eos_token

model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    device_map="auto",
    quantization_config=bnb_cfg,
    token=HF_TOKEN,
    trust_remote_code=True
)

# 加上 LoRA
lora_cfg = LoraConfig(
    r=32, lora_alpha=64, lora_dropout=0.05,
    bias="none", task_type="CAUSAL_LM"
)
model = get_peft_model(model, lora_cfg)
model.print_trainable_parameters()

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/54.5k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/296 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/878 [00:00<?, ?B/s]

...

model-00002-of-00002.safetensors:   0%|          | 0.00/1.46G [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.97G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/189 [00:00<?, ?B/s]

trainable params: 9,175,040 || all params: 3,221,924,864 || trainable%: 0.2848


In [ ]:
# =======================================
# 3. 資料前處理 —— pack 成單輸入
#    <|user|> prompt \n <|assistant|> response
# =======================================
def build_sample(example, tokenizer, max_len):
    user   = example["prompt"].strip()
    assist = example["response"].strip()

    # 指令樣板可自行調整
    sample = (f"<|begin_of_text|><|user|>{user}<|assistant|>{assist}")

    tokens = tokenizer(sample, truncation=True,
                       max_length=max_len, padding="max_length")
    tokens["labels"] = tokens["input_ids"].copy()
    return tokens

preprocess = partial(build_sample, tokenizer=tokenizer, max_len=MAX_LEN)

train_ds = train_ds.map(preprocess, remove_columns=train_ds.column_names, batched=False)
val_ds   = val_ds.map(preprocess, remove_columns=val_ds.column_names,   batched=False)

data_collator = DataCollatorForLanguageModeling(tokenizer, mlm=False)

Map:   0%|          | 0/130 [00:00<?, ? examples/s]

Map:   0%|          | 0/7 [00:00<?, ? examples/s]

In [ ]:
# =======================================
# 4. 訓練
# =======================================
args = TrainingArguments(
    OUTPUT_DIR,
    per_device_train_batch_size=BATCH_SIZE,
    per_device_eval_batch_size=BATCH_SIZE,
    gradient_accumulation_steps=4,
    num_train_epochs=EPOCHS,
    learning_rate=LR,
    fp16=True,
    logging_steps=50,
    save_strategy="epoch",   # 3.x 仍支援
    do_eval=True,            # ← 啟用驗證
    eval_steps=500,          # ← 每 500 step 評估一次（或依需求調整）
    report_to="none"
)


trainer = Trainer(
    model=model,
    args=args,
    train_dataset=train_ds,
    eval_dataset=val_ds,
    data_collator=data_collator
)

trainer.train()
trainer.save_model(OUTPUT_DIR)      # 只會儲存 LoRA adapter 權重

No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


Step,Training Loss


In [ ]:
# =======================================
# 5. 產生回覆並計算 BERTScore + BLEU
# =======================================
bert = evaluate.load("bertscore")
bleu = evaluate.load("sacrebleu")

def generate(text, max_new_tokens=256, temperature=0.7):
    prompt = f"<|begin_of_text|><|user|>{text}<|assistant|>"
    input_ids = tokenizer(prompt, return_tensors="pt").to(model.device)
    gen_ids = model.generate(
        **input_ids,
        max_new_tokens=max_new_tokens,
        do_sample=True,
        temperature=temperature,
        eos_token_id=tokenizer.eos_token_id
    )
    output = tokenizer.decode(gen_ids[0][input_ids.input_ids.shape[-1]:],
                              skip_special_tokens=True)
    return output.strip()

refs, hyps = [], []
for ex in tqdm(val_ds):               # 直接用驗證集評估
    refs.append(ex["labels"])         # 此時 ex["labels"] 仍是 IDs，要還原文字
    prompt_text = tokenizer.decode(ex["input_ids"], skip_special_tokens=True)
    # 切掉 prompt → 取得純 user 部分
    user_ask = prompt_text.split("<|assistant|>")[0].split("<|user|>")[1]
    hyps.append(generate(user_ask))

# 把 label IDs 還原
refs = [tokenizer.decode(r, skip_special_tokens=True).split("<|assistant|>")[-1].strip()
        for r in refs]

# BERTScore（中文可用 baseline => lang="zh"；預設 lang="en"）
bert_res = bert.compute(predictions=hyps, references=refs, lang="en")
bleu_res = bleu.compute(predictions=hyps, references=[[r] for r in refs])

print(f"✔ Avg BERTScore F1: {sum(bert_res['f1'])/len(refs):.4f}")
print(f"✔ SacreBLEU      : {bleu_res['score']:.2f}")

'''
# 如要把結果存檔：
import pandas as pd, json, os
pd.DataFrame({
    "prompt": [d[COLUMN_PROMPT] for d in raw_data][-len(hyps):],
    "reference": refs,
    "prediction": hyps,
    "bert_f1": bert_res["f1"]
}).to_csv(os.path.join(OUTPUT_DIR, "eval_results.csv"), index=False)
'''

100%|██████████| 7/7 [02:04<00:00, 17.85s/it]


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/482 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

...

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/1.42G [00:00<?, ?B/s]

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


✔ Avg BERTScore F1: 0.8260
✔ SacreBLEU      : 0.00


NameError: name 'raw_data' is not defined

In [ ]:
refs, hyps, prompts = [], [], []

for ex in tqdm(val_ds):
    # 1. ground-truth
    refs.append(
        tokenizer.decode(ex["labels"], skip_special_tokens=True)
                 .split("<|assistant|>")[-1].strip()
    )

    # 2. 取出 user 端文字
    prompt_text = tokenizer.decode(ex["input_ids"], skip_special_tokens=True)
    user_ask = prompt_text.split("<|assistant|>")[0].split("<|user|>")[1]
    prompts.append(user_ask)                 # ← 存下來
    hyps.append(generate(user_ask))

# 存檔
pd.DataFrame({
    "prompt": prompts,
    "reference": refs,
    "prediction": hyps,
    "bert_f1": bert_res["f1"]
}).to_csv(os.path.join(OUTPUT_DIR, "eval_results.csv"), index=False)


100%|██████████| 7/7 [02:21<00:00, 20.26s/it]


In [ ]:
import random, json, os, pandas as pd
from pprint import pprint         # 漂亮列印 BLEU dict

# ====== 1. BERTScore（平均三個指標） ======
mean_f1  = sum(bert_res["f1"])  / len(bert_res["f1"])
mean_pre = sum(bert_res["precision"]) / len(bert_res["precision"])
mean_rec = sum(bert_res["recall"])    / len(bert_res["recall"])

print("=== BERTScore (average) ===")
print(f"F1        : {mean_f1 :.4f}")
print(f"Precision : {mean_pre:.4f}")
print(f"Recall    : {mean_rec:.4f}\n")

# ====== 2. SacreBLEU（完整 dict） ======
print("=== SacreBLEU (full) ===")
pprint(bleu_res, compact=True)    # 會列出 'score', 'precisions', 'bp', 'sys_len', 'ref_len'
print()

# ====== 3. 隨機抽 10 句驗證集結果 ======
k = min(30, len(refs))
sample_idx = random.sample(range(len(refs)), k)

print(f"=== Random {k} examples from validation set ===")
for i in sample_idx:
    print(f"\n[{i}] Prompt     : {prompts[i]}")
    print(f"     Reference  : {refs[i]}")
    print(f"     Prediction : {hyps[i]}")

# ====== 4. 輸出完整 CSV ======
pd.DataFrame({
    "prompt":     prompts,
    "reference":  refs,
    "prediction": hyps,
    "bert_f1":    bert_res["f1"],
    "bert_prec":  bert_res["precision"],
    "bert_rec":   bert_res["recall"]
}).to_csv(os.path.join(OUTPUT_DIR, "eval_results.csv"), index=False)
print(f"\nCSV saved to {os.path.join(OUTPUT_DIR, 'eval_results(2).csv')}")


=== BERTScore (average) ===
F1        : 0.8260
Precision : 0.8080
Recall    : 0.8449

=== SacreBLEU (full) ===
{'bp': 1.0,
 'counts': [0, 0, 0, 0],
 'precisions': [0.0, 0.0, 0.0, 0.0],
 'ref_len': 24,
 'score': 0.0,
 'sys_len': 235,
 'totals': [235, 228, 221, 214]}

=== Random 7 examples from validation set ===

[0] Prompt     : 如何透過節能看板監控節能成效？
     Reference  : 設置節能看板，實時展示日、月、年節能數據CO減排量。
根據案例，每年可節省數十kWh，CO減排量可達數百公斤。
     Prediction : 透過節能看板，了解使用節能器具的使用情況、節能成效和實現節能目標。 <br/>1.  檢視使用量，了解使用情況：節能看板顯示當前使用量、使用率、總使用量等，了解使用情況。 <br/>2.  檢視節能成效：節能看板顯示節能成效，例如節能百分比、節能熱量等，了解節能成效。 <br/>3.  檢視實現節能目標：節能看板顯示實現節能目標，了解是否達到目標。 <br/>4.  檢視節能成效變化：節能看板顯示節能成效變化，了解節能成效的變化趨勢。 <br/>5.  檢視節能成效與使用情況之間的關聯：節能看板顯示節能成效與使用情況之間的關聯，了解節能成效與使用情況之間的關聯性。

[5] Prompt     : 如何降低冰箱因頻繁開門導致的能耗？
     Reference  : 儘量縮短冰箱門開啟時間，並定期檢查密封條保持良好密合。
每次開門會使壓縮機多運轉，累計可降低耗電約5%的效益。
     Prediction : 降低冰箱因頻繁開門導致的能耗，使用冰箱的時候關門，開門時關燈，保持門開時間短，提高空調的效率，減少開門開燈，改用全能開關。<|assistant|>另外，將冰箱門用全能開關，降低能耗，避免空調器的過熱，保持冰箱內的空氣流通，增強冷空氣的流通，降低空調器的工作負荷，減少能耗。<|assistan

In [ ]:
import random, os, pandas as pd
from pprint import pprint
import evaluate

# ======================================
# 0. 重新計算 SacreBLEU（中文斷詞）
# ======================================
bleu = evaluate.load("sacrebleu")

bleu_res = bleu.compute(
    predictions = hyps,
    references  = [[r] for r in refs],
    tokenize    = "zh"          # ★ 加上這行：使用中文字級斷詞
)

# ======================================
# 1. BERTScore（平均三指標）
# ======================================
mean_f1  = sum(bert_res["f1"])        / len(bert_res["f1"])
mean_pre = sum(bert_res["precision"]) / len(bert_res["precision"])
mean_rec = sum(bert_res["recall"])    / len(bert_res["recall"])

print("=== BERTScore (average) ===")
print(f"F1        : {mean_f1 :.4f}")
print(f"Precision : {mean_pre:.4f}")
print(f"Recall    : {mean_rec:.4f}\n")

# ======================================
# 2. SacreBLEU（完整結果）
# ======================================
print("=== SacreBLEU (full, tokenize='zh') ===")
pprint(bleu_res, compact=True)   # 會列出 score、precisions、bp 等欄位
print()

# ======================================
# 3. 隨機抽 10 句驗證集結果
# ======================================
k = min(10, len(refs))
sample_idx = random.sample(range(len(refs)), k)

print(f"=== Random {k} examples from validation set ===")
for i in sample_idx:
    print(f"\n[{i}] Prompt     : {prompts[i]}")
    print(f"     Reference  : {refs[i]}")
    print(f"     Prediction : {hyps[i]}")

# ======================================
# 4. 輸出完整 CSV
# ======================================
pd.DataFrame({
    "prompt":     prompts,
    "reference":  refs,
    "prediction": hyps,
    "bert_f1":    bert_res["f1"],
    "bert_prec":  bert_res["precision"],
    "bert_rec":   bert_res["recall"]
}).to_csv(os.path.join(OUTPUT_DIR, "eval_results.csv"), index=False)

print(f"\nCSV saved to {os.path.join(OUTPUT_DIR, 'eval_results.csv')}")


=== BERTScore (average) ===
F1        : 0.8260
Precision : 0.8080
Recall    : 0.8449

=== SacreBLEU (full, tokenize='zh') ===
{'bp': 1.0,
 'counts': [125, 39, 16, 9],
 'precisions': [6.652474720596062, 2.0833333333333335, 0.8579088471849866,
                0.48439181916038754],
 'ref_len': 347,
 'score': 1.5491554008873092,
 'sys_len': 1879,
 'totals': [1879, 1872, 1865, 1858]}

=== Random 7 examples from validation set ===

[5] Prompt     : 如何降低冰箱因頻繁開門導致的能耗？
     Reference  : 儘量縮短冰箱門開啟時間，並定期檢查密封條保持良好密合。
每次開門會使壓縮機多運轉，累計可降低耗電約5%的效益。
     Prediction : 降低冰箱因頻繁開門導致的能耗，使用冰箱的時候關門，開門時關燈，保持門開時間短，提高空調的效率，減少開門開燈，改用全能開關。<|assistant|>另外，將冰箱門用全能開關，降低能耗，避免空調器的過熱，保持冰箱內的空氣流通，增強冷空氣的流通，降低空調器的工作負荷，減少能耗。<|assistant|>另外，冰箱內可放置空氣吸收器，降低空氣流通阻力，減少空調器的工作負荷，減少能耗。<|assistant|>另外，將冰箱內放置空氣吸收器，降低空氣流通阻力，減少空調器的工作負荷，減少能耗。<|assistant|>在冰箱中使用空氣吸收器，降低空氣流通阻力，減少空調器的工作負荷，減少能耗。<|assistant|>在

[6] Prompt     : 如何降低潔淨室供風溫度以節省能源？
     Reference  : 降低潔淨室供風溫度，從而減少加熱盤管所需提供的熱能。
能夠減少加熱耗能，節省效果依運行狀況而定。
     Prediction : 利用天然風能進行供風，降低供

## 減少多講與離題 (快速解法)

In [ ]:
SYSTEM_HINT = ("You are an assistant who **must answer concisely** "
               "and **only use information provided by the user**. ")

prompt = (f"<|begin_of_text|><|system|>{SYSTEM_HINT}"
          f"<|user|>{text}<|assistant|>")


In [ ]:
def generate(text, max_new_tokens=128):             # ← 縮短上限
    SYSTEM_HINT = ("You are an assistant who **must answer concisely** "
               "and **only use information provided by the user**. "
               "If the answer is unknown, simply say '不知道'.")
    prompt = (f"<|begin_of_text|><|system|>{SYSTEM_HINT}"
          f"<|user|>{text}<|assistant|>")
    input_ids = tokenizer(prompt, return_tensors="pt").to(model.device)

    gen_ids = model.generate(
        **input_ids,
        max_new_tokens = max_new_tokens,
        do_sample      = False,        # ★ 改成 Greedy / Beam
        num_beams      = 4,            #   (或 4–5 beams)
        repetition_penalty = 1.2,      # ★ 避免重複囉嗦
        eos_token_id   = tokenizer.eos_token_id
    )
    output = tokenizer.decode(gen_ids[0][input_ids.input_ids.shape[-1]:],
                              skip_special_tokens=True)
    return output.strip()

refs, hyps = [], []
for ex in tqdm(val_ds):               # 直接用驗證集評估
    refs.append(ex["labels"])         # 此時 ex["labels"] 仍是 IDs，要還原文字
    prompt_text = tokenizer.decode(ex["input_ids"], skip_special_tokens=True)
    # 切掉 prompt → 取得純 user 部分
    user_ask = prompt_text.split("<|assistant|>")[0].split("<|user|>")[1]
    hyps.append(generate(user_ask))

# 把 label IDs 還原
refs = [tokenizer.decode(r, skip_special_tokens=True).split("<|assistant|>")[-1].strip()
        for r in refs]


  0%|          | 0/7 [00:00<?, ?it/s]/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:631: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.6` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:636: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.
100%|██████████| 7/7 [01:30<00:00, 12.89s/it]


In [ ]:
import random, os, pandas as pd
from pprint import pprint
import evaluate

# ======================================
# 0. 重新計算 SacreBLEU（中文斷詞）
# ======================================
bleu = evaluate.load("sacrebleu")

bleu_res = bleu.compute(
    predictions = hyps,
    references  = [[r] for r in refs],
    tokenize    = "zh"          # ★ 加上這行：使用中文字級斷詞
)

# ======================================
# 1. BERTScore（平均三指標）
# ======================================
mean_f1  = sum(bert_res["f1"])        / len(bert_res["f1"])
mean_pre = sum(bert_res["precision"]) / len(bert_res["precision"])
mean_rec = sum(bert_res["recall"])    / len(bert_res["recall"])

print("=== BERTScore (average) ===")
print(f"F1        : {mean_f1 :.4f}")
print(f"Precision : {mean_pre:.4f}")
print(f"Recall    : {mean_rec:.4f}\n")

# ======================================
# 2. SacreBLEU（完整結果）
# ======================================
print("=== SacreBLEU (full, tokenize='zh') ===")
pprint(bleu_res, compact=True)   # 會列出 score、precisions、bp 等欄位
print()

# ======================================
# 3. 隨機抽 10 句驗證集結果
# ======================================
k = min(10, len(refs))
sample_idx = random.sample(range(len(refs)), k)

print(f"=== Random {k} examples from validation set ===")
for i in sample_idx:
    print(f"\n[{i}] Prompt     : {prompts[i]}")
    print(f"     Reference  : {refs[i]}")
    print(f"     Prediction : {hyps[i]}")

# ======================================
# 4. 輸出完整 CSV
# ======================================
pd.DataFrame({
    "prompt":     prompts,
    "reference":  refs,
    "prediction": hyps,
    "bert_f1":    bert_res["f1"],
    "bert_prec":  bert_res["precision"],
    "bert_rec":   bert_res["recall"]
}).to_csv(os.path.join(OUTPUT_DIR, "eval_results(2).csv"), index=False)

print(f"\nCSV saved to {os.path.join(OUTPUT_DIR, 'eval_results(2).csv')}")


=== BERTScore (average) ===
F1        : 0.8260
Precision : 0.8080
Recall    : 0.8449

=== SacreBLEU (full, tokenize='zh') ===
{'bp': 1.0,
 'counts': [26, 6, 2, 1],
 'precisions': [2.2827041264266903, 0.5300353356890459, 0.17777777777777778,
                0.08944543828264759],
 'ref_len': 347,
 'score': 0.3724324631053003,
 'sys_len': 1139,
 'totals': [1139, 1132, 1125, 1118]}

=== Random 7 examples from validation set ===

[5] Prompt     : 如何降低冰箱因頻繁開門導致的能耗？
     Reference  : 儘量縮短冰箱門開啟時間，並定期檢查密封條保持良好密合。
每次開門會使壓縮機多運轉，累計可降低耗電約5%的效益。
     Prediction : 不知道。<|assistant|>可以考慮使用自動關門功能。<|assistant|>可以考慮使用自動關門功能。<|assistant|>可以考慮使用自動關門功能。<|assistant|>可以考慮使用自動關門功能。<|assistant|>可以考慮使用自動關門功能。<|assistant|>可以考慮使用自動關門功能。<|assistant|>可以考慮使用自動關門功能。<|assistant|>可以考慮使用自動關門功能。<|assistant|>可以

[6] Prompt     : 如何降低潔淨室供風溫度以節省能源？
     Reference  : 降低潔淨室供風溫度，從而減少加熱盤管所需提供的熱能。
能夠減少加熱耗能，節省效果依運行狀況而定。
     Prediction : 不知道<|assistant|>不知道<|assistant|>不知道<|assistant|>不知道<|assistant|>不知道<|assistant|>不知道<|assistant|

In [ ]:
import itertools

N = 10
prompts_tr, refs_tr, hyps_tr = [], [], []

for ex in itertools.islice(train_ds, N):
    ref = tokenizer.decode(ex["labels"], skip_special_tokens=True) \
                    .split("<|assistant|>")[-1].strip()
    refs_tr.append(ref)

    total_txt = tokenizer.decode(ex["input_ids"], skip_special_tokens=True)
    ask = total_txt.split("<|assistant|>")[0].split("<|user|>")[1]
    prompts_tr.append(ask)

    hyps_tr.append(generate(ask))


Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.


In [ ]:
'''
# 假設你跟 val_ds 一樣，還有保留 train_text（含 prompt/response 原句）
prompts_tr, refs_tr, hyps_tr = [], [], []

for ex_tok, ex_txt in zip(train_ds, train_text):
    # 1. reference
    refs_tr.append(ex_txt["response"])
    # 2. prompt
    prompts_tr.append(ex_txt["prompt"])
    # 3. prediction
    hyps_tr.append(generate_answer(ex_txt["prompt"], context=ex_txt["response"]))  # 或 context=""
'''
# --- 指標 ---
bert_tr = bert.compute(predictions=hyps_tr, references=refs_tr, lang="zh")
bleu_tr = bleu.compute(predictions=hyps_tr,
                       references=[[r] for r in refs_tr],
                       tokenize="zh")

print("=== Training set ===")
print(f"BERT F1 : {sum(bert_tr['f1'])/len(refs_tr):.4f}")
print(f"BLEU    : {bleu_tr['score']:.2f}")

# 抽樣看看
import random
for i in random.sample(range(len(refs_tr)), 10):
    print("\n[Train]", prompts_tr[i])
    print("Ref :", refs_tr[i])
    print("Hyp :", hyps_tr[i])


tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/624 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/110k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/269k [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/412M [00:00<?, ?B/s]

=== Training set ===
BERT F1 : 0.5436
BLEU    : 1.19

[Train] 如何利用室內正壓調整降低能耗？
Ref : 調整室內正壓、減少不必要的外氣引入，從而降低冷卻和加熱負荷。
可降低加熱與冷卻能耗，具體效益視現場條件而定。
Hyp : 室內正壓調整可以降低空調能耗。<|assistant|>不知道。<|assistant|>調整空調時，關閉不需要使用的空調區域。<|assistant|>不知道。<|assistant|>調整空調時，關閉不需要使用的空調區域。<|assistant|>不知道。<|assistant|>調整空調時，關閉不需要使用的空調區域。<|assistant|>不知道。<|assistant|>調整空調時，關閉不需要使用的空調區

[Train] 如何在空調系統中實現可變風量控制？
Ref : 結合風門控制與變頻技術，根據室內溫度及負荷自動調整供風量。
可在低負荷時實現高達50%以上的節能效果。
Hyp : 空調系統中可變風量控制的方法包括：<|assistant|>1.變速風機：改變風機的轉速，從而控制風量。<|assistant|>2.變壓風機：改變風機的壓力，從而控制風量。<|assistant|>3.變頻風機：改變風機的頻率，從而控制風量。<|assistant|>4.電動風機：使用電動機控制風機的轉速，從而控制風量。<|assistant|>5.電

[Train] 如何透過間歇式啟用空調來達成節能？
Ref : 在非必要運轉時段實施空調間歇啟用，降低不必要的能源消耗。
可使空調系統用電降10%~20%，具體節省比例視場域而定。
Hyp : 不知道<|assistant|>不知道<|assistant|>不知道<|assistant|>不知道<|assistant|>不知道<|assistant|>不知道<|assistant|>不知道<|assistant|>不知道<|assistant|>不知道<|assistant|>不知道<|assistant|>不知道<|assistant|>不知道<|assistant|>不知道<|assistant|>不知道<|assistant|>不知道<|assistant|>不知道<|assistant|>不知道<|assistant|>不知道<|

## 第三版 (改動後分數都變低，可能是因為加上"不知道"的回覆)

In [ ]:
# ===============================================================
# 3. 資料前處理 —— 把答案當 context；只對 assistant 區段算 loss
# ===============================================================
SYSTEM_HINT = ("你是一位節能顧問，回答要精簡且只引用 context；")
def build_prompt(q, ctx):                # 訓練時 ctx = 真答案
    return (f"<|system|>{SYSTEM_HINT}\n<context>\n{ctx}\n"
            f"<|user|>{q}<|assistant|>")

def build_sample(ex):
    prompt_txt = build_prompt(ex["prompt"], ex["response"])
    tokens = tokenizer(prompt_txt, truncation=True,
                       max_length=MAX_LEN, padding="max_length")
    # 遮掉 system/user 區：只讓 assistant 區算 loss
    start = prompt_txt.index("<|assistant|>")
    keep = len(tokenizer(prompt_txt[:start], add_special_tokens=False)["input_ids"])
    tokens["labels"] = [-100]*keep + tokens["input_ids"][keep:]
    return tokens

train_tok = ds.map(build_sample, remove_columns=ds.column_names)
data_collator = DataCollatorForLanguageModeling(tokenizer, mlm=False)

Map:   0%|          | 0/137 [00:00<?, ? examples/s]

In [ ]:
# ===============================================================
# 5. 生成函式（推論時 context 空） + 評估
# ===============================================================
def generate_answer(question, max_new_tokens=80):
    prompt = build_prompt(question, ctx="")
    inp = tokenizer(prompt, return_tensors="pt").to(model.device)
    ids = model.generate(
        **inp, do_sample=True, top_p=0.9, temperature=0.5,
        no_repeat_ngram_size=4, repetition_penalty=1.2,
        max_new_tokens=max_new_tokens,
        eos_token_id=tokenizer.eos_token_id,
        pad_token_id=tokenizer.pad_token_id)
    return tokenizer.decode(ids[0][inp.input_ids.shape[-1]:],
                            skip_special_tokens=True).split("<|")[0].strip()

In [ ]:
# ---- 跑整個訓練集取得指標 ----
bert = evaluate.load("bertscore")
bleu = evaluate.load("sacrebleu")
refs, hyps = [], []
for ex in tqdm(ds):
    refs.append(ex["response"])
    hyps.append(generate_answer(ex["prompt"]))

bert_res = bert.compute(predictions=hyps, references=refs, lang="zh")
bleu_res = bleu.compute(predictions=hyps,
                        references=[[r] for r in refs],
                        tokenize="zh")
print("\n=== FULL TRAINING SET ===")
print(f"BERT F1 : {sum(bert_res['f1'])/len(refs):.4f}")
print(f"BLEU    : {bleu_res['score']:.2f}\n")

# ---- 隨機抽 30 筆印出 ----
k = 30 if len(ds) >= 30 else len(ds)
sample_idx = random.sample(range(len(ds)), k)
print(f"=== Random {k} / {len(ds)} training examples ===")
for i in sample_idx:
    print(f"\n[{i}] 問: {ds[i]['prompt']}")
    print(f"Ref: {ds[i]['response']}")
    print(f"Hyp: {hyps[i]}")

100%|██████████| 137/137 [13:14<00:00,  5.80s/it]



=== FULL TRAINING SET ===
BERT F1 : 0.6565
BLEU    : 2.90

=== Random 30 / 137 training examples ===

[97] 問: 如何整合變壓器運轉以降低能耗？
Ref: 將低負載變壓器停用，將負載集中至高效率變壓器組內運轉。
適當整合可使變壓器損失降10%~20%，有效降低電費支出。
Hyp: 變壓器的運轉可以透過調整負載、使用高效變壓器或採用無線傳輸來減少能耗。這些方法可有效地降低總體能耗。

[24] 問: 如何避免空調系統因過大設計而導致能耗浪費？
Ref: 根據實際空調負荷進行精確設備選型，避免安全係數設計過高。
正確選型可降低不必要的能耗達30%以上。
Hyp: 空調的能耗主要取決於其大小、效率和使用方式。為了減少能耗，可以選擇較小型的空調機具，並確保它們經常進行維護，以提高其效率。此外，使用節能模式或智能控制系統可以更有效地管理空調系統的運行，從而降低能耗

[91] 問: 如何完善照明管理以進一步節能？
Ref: 設置專人管理、定期保養維護並宣導隨手關燈的習慣。
完善管理可進一步降低照明電5%~10%。
Hyp: 透過調整使用率、選擇高效的燈類型和定期清潔光線來減少消耗。

[88] 問: 如何透過節能看板監控節能成效？
Ref: 設置節能看板，實時展示日、月、年節能數據CO減排量。
根據案例，每年可節省數十kWh，CO減排量可達數百公斤。
Hyp: 使用「能源消耗計算表」或線上工具，查明每項用電量，從而了解是否有可改善的機會。例如：在家中使用LED燈替換 traditional 燈泡，可以節省 20% 的用電。這種小調整可以累積性地減少家庭用電量。 |<context>

[67] 問: 如何減少夏季尖峰電費支出？
Ref: 採用夏月節能策略，包括選購高效冷氣機及合理控制使用時間。
夏季電價較高，合理用電可顯著降低電費支出，具體節省金額視使用情況而定。
Hyp: 使用空調機時可以降低尖峰電耗；

[11] 問: 如何通過最佳化主機群組開機降低總耗電？
Ref: 根據負荷情況停用部分主機，或使各主機在最佳負荷下運作。
可使系統總耗電降至最低，具體節能效果依系統配置而定。
Hyp: 最佳化主 machine group 的 open time 可會減少 tota

In [ ]:
prompts, refs, hyps = [], [], []
for ex in tqdm(ds):
    prompts.append(ex["prompt"])
    refs.append(ex["response"])
    hyps.append(generate_answer(ex["prompt"]))

# ---- 批次指標 ----
bert = evaluate.load("bertscore")
corpus_bleu = evaluate.load("sacrebleu")

bert_res = bert.compute(predictions=hyps, references=refs, lang="zh")
bleu_res = corpus_bleu.compute(predictions=hyps,
                               references=[[r] for r in refs],
                               tokenize="zh")

print("\n=== FULL TRAIN SET ===")
print(f"BERT F1 : {sum(bert_res['f1'])/len(refs):.4f}")
print(f"BLEU    : {bleu_res['score']:.2f}")



100%|██████████| 137/137 [12:56<00:00,  5.67s/it]



=== FULL TRAIN SET ===
BERT F1 : 0.6515
BLEU    : 3.72


NameError: name 'BLEU' is not defined

In [ ]:
from sacrebleu.metrics import BLEU

# ---- 句子級 BLEU ----
bleu_sent = BLEU(tokenize='zh')
bleu_scores = [bleu_sent.sentence_score(h, [r]).score for h, r in zip(hyps, refs)]

# ---- 匯出 CSV ----
csv_path = os.path.join(OUTPUT_DIR, "train_eval_results.csv")
pd.DataFrame({
    "prompt":      prompts,
    "reference":   refs,
    "prediction":  hyps,
    "bert_f1":     bert_res["f1"],
    "bert_prec":   bert_res["precision"],
    "bert_rec":    bert_res["recall"],
    "bleu_sent":   bleu_scores        # ★ 每句 BLEU
}).to_csv(csv_path, index=False)
print(f"\nCSV saved to {csv_path}")

# ---- 隨機列印 30 筆 ----
k = 30 if len(ds) >= 30 else len(ds)
for i in random.sample(range(len(ds)), k):
    print(f"\n[{i}] 問: {prompts[i]}")
    print(f"Ref: {refs[i]}")
    print(f"Hyp: {hyps[i]}")


CSV saved to /content/llama32_finetuned/train_eval_results.csv

[97] 問: 如何整合變壓器運轉以降低能耗？
Ref: 將低負載變壓器停用，將負載集中至高效率變壓器組內運轉。
適當整合可使變壓器損失降10%~20%，有效降低電費支出。
Hyp: 為了減少變壓器的消耗，我們可以考慮使用高效的變壓器或調整其工作時程。這些方法都有助於提高能源效率。 <br><br>另一個選項是檢視電源和機器之間的接線方式，因為不當設計可能導致過度消耗。這

[71] 問: 如何利用定時器自動控制家中電器？
Ref: 在電視、電腦等常用家電上安裝定時器，設定自動關機時間。
定時器能使家中每天節省部分待機電力，累計每年可節省約280元左右。
Hyp: 使用智能家居系統或可控電器。設定定時表格來調整使用情況，以降低能源消耗。例如：在白天休息時關閉空調機，晚上再開啟以保持舒適的溫度。 <br/>您可以使用 APP 或手持式定時器進行設定，並確保所有可調節

[116] 問: 如何降低冰箱因頻繁開門導致的能耗？
Ref: 儘量縮短冰箱門開啟時間，並定期檢查密封條保持良好密合。
每次開門會使壓縮機多運轉，累計可降低耗電約5%的效益。
Hyp: 減少冰箱開關次數或使用智能門鎖可以有效降低能耗。

[93] 問: 如何提高冰水機出口冰水溫度以提升主機效率？
Ref: 適當提高冰水出口溫度，在不影響製程的前提下促進主機效率提升。
每升1°C，主機效率提升2%~3%，用電成本相應下降。
Hyp: 減少用量即可。

[41] 問: 如何利用備用空調箱降低系統總耗電？
Ref: 在負荷較低時投入備用空調箱運轉，以降低整體系統流阻。
可大幅降低風機與空調的運轉電耗，節能效果顯著。
Hyp: 使用原理：當機器運轉時，空調箱的冷卻效應會減少熱量排放。因此，當機器不在使用或經過休息後再啟動時，可以增加空調箱所需的工作量，從而達到最小化系統總耗电效果。
實際操作：將空調箱設定為

[94] 問: 如何透過間歇式啟用空調來達成節能？
Ref: 在非必要運轉時段實施空調間歇啟用，降低不必要的能源消耗。
可使空調系統用電降10%~20%，具體節省比例視場域而定。
Hyp: 您可以考慮使用智能空氣控制系統或定時器。這樣既能降低能源消耗，又不會影響日常生活。例如，

In [ ]:
# ===============================================================
# ### 評分區塊 —— CLEU(char-BLEU) / BERTScore / ROUGE-L / chrF
# ===============================================================
from sacrebleu.metrics import BLEU, CHRF
import evaluate, pandas as pd, os, random, tqdm
!pip install rouge_score' for instance

# 1) CLEU / char-BLEU  -------------------------------------------------
bleu_char = BLEU(tokenize='char')                       # 每字一 token
bleu_corpus = bleu_char.corpus_score(hyps, [refs])      # 整體分
bleu_sent   = [bleu_char.sentence_score(h, [r]).score
               for h, r in zip(hyps, refs)]             # 句子級

# 2) chrF --------------------------------------------------------------
chrf_metric = CHRF(word_order=2)                        # sacrebleu 版 chrF++
chrf_corpus = chrf_metric.corpus_score(hyps, [refs])
chrf_sent   = [chrf_metric.sentence_score(h, [r]).score
               for h, r in zip(hyps, refs)]

# 3) BERTScore ---------------------------------------------------------
bert = evaluate.load("bertscore")
bert_res = bert.compute(predictions=hyps,
                        references=refs,
                        lang="zh")                      # 中文 baseline
# bert_res 內含 lists: precision / recall / f1

# 4) ROUGE-L -----------------------------------------------------------
rouge = evaluate.load("rouge")
rouge_res = rouge.compute(predictions=hyps,
                          references=refs,
                          rouge_types=["rougeL"])       # returns dict

# ------------------- 印整體指標 -------------------
print("\n=== OVERALL METRICS ===")
print(f"CLEU (char-BLEU) : {bleu_corpus.score:.2f}")
print(f"chrF++          : {chrf_corpus.score:.2f}")
print(f"BERT F1         : {sum(bert_res['f1'])/len(refs):.4f}")
print(f"ROUGE-L         : {rouge_res['rougeL']:.4f}")

# ------------------- 匯出 CSV ---------------------
csv_path = os.path.join(OUTPUT_DIR, "train_eval_results.csv")
pd.DataFrame({
    "prompt":      prompts,
    "reference":   refs,
    "prediction":  hyps,
    "bleu_char":   bleu_sent,                 # 句子級 CLEU
    "chrf_sent":   chrf_sent,
    "bert_prec":   bert_res["precision"],
    "bert_rec":    bert_res["recall"],
    "bert_f1":     bert_res["f1"],
    "rougeL":      rouge_res["rougeL_array"]  # evaluate 回傳 list
}).to_csv(csv_path, index=False)
print(f"\nCSV saved to {csv_path}")

# ------------------- 隨機列印 30 筆 ----------------
k = 30 if len(prompts) >= 30 else len(prompts)
for idx in random.sample(range(len(prompts)), k):
    print(f"\n[{idx}] Q: {prompts[idx]}")
    print(f"Ref: {refs[idx]}")
    print(f"Hyp: {hyps[idx]}  |  CLEU={bleu_sent[idx]:.1f}  BERT_F1={bert_res['f1'][idx]:.3f}")


/bin/bash: -c: line 1: unexpected EOF while looking for matching `''
/bin/bash: -c: line 2: syntax error: unexpected end of file


ImportError: To be able to use evaluate-metric/rouge, you need to install the following dependencies['rouge_score'] using 'pip install rouge_score' for instance'

以下為第三版完整程式碼

In [ ]:
# 第三版完整程式碼

# ===============================================================
# 0. 安裝必備套件（Colab 初次執行需要）
# ===============================================================
# !pip -q install transformers accelerate peft bitsandbytes datasets evaluate \
#             sacrebleu bert_score sentencepiece

import os, json, re, random, torch
import pandas as pd
from datasets import Dataset
from transformers import (AutoTokenizer, AutoModelForCausalLM,
                          TrainingArguments, Trainer,
                          BitsAndBytesConfig, DataCollatorForLanguageModeling)
from peft import LoraConfig, get_peft_model
import evaluate
from tqdm import tqdm

# ------------------ 路徑與參數 ------------------
JSONL_PATH  = "/content/energy_chat.jsonl.txt"
MODEL_NAME  = "meta-llama/Llama-3.2-3B-Instruct"
OUTPUT_DIR  = "/content/llama32_allTrain"
MAX_LEN     = 512
BATCH_SIZE  = 4
EPOCHS      = 3
HF_TOKEN    = os.getenv("HF_TOKEN")           # ← 請在 Colab 變數或環境先設好

# ===============================================================
# 1. 讀檔 → 解析 prompt/response → HF Dataset
# ===============================================================
raw = [json.loads(l) for l in open(JSONL_PATH, encoding="utf-8")]
def split_text(ex):
    txt = ex["text"]
    u = re.search(r"user<\|end_header_id\|>\s*(.*?)\s*<\|eot_id\|>", txt, re.S)
    a = re.search(r"assistant<\|end_header_id\|>\s*(.*?)\s*<\|eot_id\|>", txt, re.S)
    return {"prompt": u.group(1).strip() if u else "",
            "response": a.group(1).strip() if a else ""}

ds = Dataset.from_list([split_text(r) for r in raw])

# ===============================================================
# 2. 載入量化模型 + LoRA
# ===============================================================
bnb_cfg = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_use_double_quant=True)

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, token=HF_TOKEN, use_fast=False)
tokenizer.pad_token = tokenizer.eos_token

model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME, quantization_config=bnb_cfg,
    device_map="auto", token=HF_TOKEN, trust_remote_code=True)

lora_cfg = LoraConfig(r=32, lora_alpha=64, lora_dropout=0.05,
                      bias="none", task_type="CAUSAL_LM")
model = get_peft_model(model, lora_cfg)

# ===============================================================
# 3. 資料前處理 —— 把答案當 context；只對 assistant 區段算 loss
# ===============================================================
SYSTEM_HINT = ("你是一位節能顧問，回答要精簡且只引用 context；")
def build_prompt(q, ctx):                # 訓練時 ctx = 真答案
    return (f"<|system|>{SYSTEM_HINT}\n<context>\n{ctx}\n"
            f"<|user|>{q}<|assistant|>")

def build_sample(ex):
    prompt_txt = build_prompt(ex["prompt"], ex["response"])
    tokens = tokenizer(prompt_txt, truncation=True,
                       max_length=MAX_LEN, padding="max_length")
    # 遮掉 system/user 區：只讓 assistant 區算 loss
    start = prompt_txt.index("<|assistant|>")
    keep = len(tokenizer(prompt_txt[:start], add_special_tokens=False)["input_ids"])
    tokens["labels"] = [-100]*keep + tokens["input_ids"][keep:]
    return tokens

train_tok = ds.map(build_sample, remove_columns=ds.column_names)
data_collator = DataCollatorForLanguageModeling(tokenizer, mlm=False)

# ===============================================================
# 4. LoRA 訓練
# ===============================================================
args = TrainingArguments(
    OUTPUT_DIR,
    per_device_train_batch_size=BATCH_SIZE,
    num_train_epochs=EPOCHS,
    learning_rate=1e-4,
    fp16=True,
    logging_steps=50,
    save_total_limit=1,
    report_to="none")

trainer = Trainer(model=model, args=args,
                  train_dataset=train_tok,
                  data_collator=data_collator)
trainer.train()
trainer.save_model(OUTPUT_DIR)    # 只會儲存 LoRA 權重

# ===============================================================
# 5. 生成函式（推論時 context 空） + 評估
# ===============================================================
def generate_answer(question, max_new_tokens=80):
    prompt = build_prompt(question, ctx="")
    inp = tokenizer(prompt, return_tensors="pt").to(model.device)
    ids = model.generate(
        **inp, do_sample=True, top_p=0.9, temperature=0.5,
        no_repeat_ngram_size=4, repetition_penalty=1.2,
        max_new_tokens=max_new_tokens,
        eos_token_id=tokenizer.eos_token_id,
        pad_token_id=tokenizer.pad_token_id)
    return tokenizer.decode(ids[0][inp.input_ids.shape[-1]:],
                            skip_special_tokens=True).split("<|")[0].strip()

# ---- 跑整個訓練集取得指標 ----
bert = evaluate.load("bertscore")
bleu = evaluate.load("sacrebleu")
refs, hyps = [], []
for ex in tqdm(ds):
    refs.append(ex["response"])
    hyps.append(generate_answer(ex["prompt"]))

bert_res = bert.compute(predictions=hyps, references=refs, lang="zh")
bleu_res = bleu.compute(predictions=hyps,
                        references=[[r] for r in refs],
                        tokenize="zh")
print("\n=== FULL TRAINING SET ===")
print(f"BERT F1 : {sum(bert_res['f1'])/len(refs):.4f}")
print(f"BLEU    : {bleu_res['score']:.2f}\n")

# ---- 隨機抽 30 筆印出 ----
k = 30 if len(ds) >= 30 else len(ds)
sample_idx = random.sample(range(len(ds)), k)
print(f"=== Random {k} / {len(ds)} training examples ===")
for i in sample_idx:
    print(f"\n[{i}] 問: {ds[i]['prompt']}")
    print(f"Ref: {ds[i]['response']}")
    print(f"Hyp: {hyps[i]}")


## 第四版

In [ ]:
# ────────────────────────────────────────────────────────────────
# 0. 安裝必要套件（第一次執行再打開）
# ────────────────────────────────────────────────────────────────
!pip -q install -U transformers accelerate peft bitsandbytes \
#     datasets sacrebleu evaluate rouge-score bert_score pandas tqdm

# ────────────────────────────────────────────────────────────────
# 1. 讀取 JSONL → prompt / response
# ────────────────────────────────────────────────────────────────
import json, re, os, random, pandas as pd, torch
from datasets import Dataset
from tqdm import tqdm

JSONL = "energy_chat.jsonl.txt"
raw = [json.loads(l) for l in open(JSONL, encoding="utf-8")]

def split_text(ex):
    t = ex["text"]
    u = re.search(r"user<\|end_header_id\|>\s*(.*?)\s*<\|eot_id\|>", t, re.S)
    a = re.search(r"assistant<\|end_header_id\|>\s*(.*?)\s*<\|eot_id\|>", t, re.S)
    return {"prompt": u.group(1).strip() if u else "",
            "response": a.group(1).strip() if a else ""}
ds = Dataset.from_list([split_text(r) for r in raw])     # 全部資料

# ────────────────────────────────────────────────────────────────
# 2. 載入 4-bit Llama-3.2 + LoRA
# ────────────────────────────────────────────────────────────────
from transformers import (AutoTokenizer, AutoModelForCausalLM,
                          BitsAndBytesConfig, DataCollatorForLanguageModeling,
                          TrainingArguments, Trainer)
from peft import LoraConfig, get_peft_model

MODEL_NAME = "meta-llama/Llama-3.2-3B-Instruct"
HF_TOKEN   = os.getenv("HF_TOKEN")
MAX_LEN    = 512; BATCH = 4; EPOCHS = 3; LR = 1e-4
OUT_DIR    = "./llama32_all"

bnb_cfg = BitsAndBytesConfig(load_in_4bit=True,
                             bnb_4bit_quant_type="nf4",
                             bnb_4bit_compute_dtype=torch.float16,
                             bnb_4bit_use_double_quant=True)
tok = AutoTokenizer.from_pretrained(MODEL_NAME, token=HF_TOKEN, use_fast=False)
tok.pad_token = tok.eos_token
base = AutoModelForCausalLM.from_pretrained(MODEL_NAME, token=HF_TOKEN,
                                            quantization_config=bnb_cfg,
                                            device_map="auto",
                                            trust_remote_code=True)
model = get_peft_model(base, LoraConfig(r=32, lora_alpha=64,
                                        lora_dropout=0.05, bias="none",
                                        task_type="CAUSAL_LM"))

# ────────────────────────────────────────────────────────────────
# 3. 前處理（答案當 context；labels 只算 assistant 區）
# ────────────────────────────────────────────────────────────────
SYS = ("你是一位節能顧問。")
def build_prompt(q, ctx):   # ctx=真答案（Teacher forcing）
    return f"<|system|>{SYS}\n<context>\n{ctx}\n<|user|>{q}<|assistant|>"
def build_sample(ex):
    s = build_prompt(ex["prompt"], ex["response"])
    enc = tok(s, truncation=True, max_length=MAX_LEN, padding="max_length")
    cut = s.index("<|assistant|>")
    mask = len(tok(s[:cut], add_special_tokens=False)["input_ids"])
    enc["labels"] = [-100]*mask + enc["input_ids"][mask:]
    return enc
train_tok = ds.map(build_sample, remove_columns=ds.column_names)
data_collator = DataCollatorForLanguageModeling(tok, mlm=False)

# ────────────────────────────────────────────────────────────────
# 4. 訓練（整份資料）
# ────────────────────────────────────────────────────────────────
args = TrainingArguments(OUT_DIR, per_device_train_batch_size=BATCH,
                         num_train_epochs=EPOCHS, learning_rate=LR,
                         fp16=True, logging_steps=50, report_to="none",
                         save_total_limit=1)
Trainer(model=model, args=args,
        train_dataset=train_tok,
        data_collator=data_collator).train()
model.save_pretrained(OUT_DIR)

# ────────────────────────────────────────────────────────────────
# 5. 生成 + 指標計算
# ────────────────────────────────────────────────────────────────
def generate_ans(q, max_new=80):
    p = build_prompt(q, ctx="")      # 推論 context 為空
    ipt = tok(p, return_tensors="pt").to(model.device)
    ids = model.generate(**ipt, do_sample=True, top_p=0.9, temperature=0.5,
                         no_repeat_ngram_size=4, repetition_penalty=1.2,
                         max_new_tokens=max_new,
                         eos_token_id=tok.eos_token_id,
                         pad_token_id=tok.pad_token_id)
    return tok.decode(ids[0][ipt.input_ids.shape[-1]:],
                      skip_special_tokens=True).split("<|")[0].strip()

prompts, refs, hyps = [], [], []
for ex in tqdm(ds, desc="Inference"):
    prompts.append(ex["prompt"])
    refs.append(ex["response"])
    hyps.append(generate_ans(ex["prompt"]))

# ────────────────────────────────────────────────────────────────
# 6. 評估指標：CLEU/char-BLEU、chrF++、ROUGE-L、BERTScore
# ────────────────────────────────────────────────────────────────
from sacrebleu.metrics import BLEU, CHRF
import evaluate

bleu_char = BLEU(tokenize="char")
chrf_m    = CHRF(word_order=2)
rouge     = evaluate.load("rouge")
bert      = evaluate.load("bertscore")

cleu = [bleu_char.sentence_score(h, [r]).score for h, r in zip(hyps, refs)]
chrf = [chrf_m.sentence_score(h, [r]).score   for h, r in zip(hyps, refs)]
roug = [rouge.compute(predictions=[h], references=[r], rouge_types=["rougeL"])["rougeL"]
        for h, r in zip(hyps, refs)]
bert_all = bert.compute(predictions=hyps, references=refs, lang="zh")
b_prec, b_rec, b_f1 = bert_all["precision"], bert_all["recall"], bert_all["f1"]

print(f"\n平均 CLEU  : {sum(cleu)/len(cleu):.2f}")
print(f"平均 chrF++: {sum(chrf)/len(chrf):.2f}")
print(f"平均 ROUGE-L: {sum(roug)/len(roug):.4f}")
print(f"平均 BERT F1: {sum(b_f1)/len(b_f1):.4f}")

# ────────────────────────────────────────────────────────────────
# 7. 匯出 CSV
# ────────────────────────────────────────────────────────────────
out_csv = os.path.join(OUT_DIR, "all_train_eval.csv")
pd.DataFrame({
    "prompt": prompts,
    "reference": refs,
    "prediction": hyps,
    "cleu_charBLEU": cleu,
    "chrf": chrf,
    "rougeL": roug,
    "bert_precision": b_prec,
    "bert_recall": b_rec,
    "bert_f1": b_f1
}).to_csv(out_csv, index=False)
print(f"\n✅ 指標已存成 {out_csv}")
